<div style="
background-color:#f1f8e9;
padding:28px;
border-radius:12px;
border-left:8px solid #689f38;
font-family:Segoe UI, Arial, sans-serif;
">

<h1 style="color:#689f38; margin:0 0 10px 0; font-weight:700;">
Tiền xử lý dữ liệu (Data Preprocessing)
</h1>

<p style="margin:0 0 10px 0; font-size:16px; color:#111;">
<b>Mục tiêu:</b> Thực hiện các bước preprocessing cơ bản để chuẩn bị dữ liệu sạch,
đảm bảo chất lượng và consistency cho các phân tích tiếp theo.
</p>

<p style="margin:0 0 12px 0; font-size:14px; color:#333;">
<b>Input:</b> healthcare-dataset-stroke-data.csv (raw data)<br>
<b>Các bước:</b> Drop ID, Chuẩn hóa text, Xử lý unknown values<br>
<b>Output:</b> healthcare_cleaned.csv (11 columns, 5,110 rows)
</p>

</div>

In [1]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("✓ Đã import thư viện thành công!")

✓ Đã import thư viện thành công!


In [2]:
# Load dữ liệu gốc
file_path = '../data/healthcare-dataset-stroke-data.csv'
df_raw = pd.read_csv(file_path)

print(f"✓ Đã load dữ liệu thành công!")
print(f"Shape: {df_raw.shape[0]} rows × {df_raw.shape[1]} columns")
print(f"\nColumns: {df_raw.columns.tolist()}")

✓ Đã load dữ liệu thành công!
Shape: 5110 rows × 12 columns

Columns: ['id', 'gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status', 'stroke']


# Basic Data Preprocessing
**Mục tiêu:** Thực hiện các bước preprocessing cơ bản để chuẩn bị dữ liệu sạch cho modeling.

**Các bước thực hiện:**
1. Loại bỏ identifier column (id)
2. Chuẩn hóa text cho categorical (lowercase, strip)
3. Xử lý "unknown" trong smoking_status
4. Lưu dữ liệu đã cleaned

## Bước 1: Loại bỏ Identifier (id column)
**Purpose:** 
- Cột `id` chỉ là định danh, không mang giá trị dự báo.
- Giữ lại trong `df_raw`, loại bỏ khỏi `df_cleaned`.

**Reasoning:**
- Tránh mô hình học nhầm pattern từ số định danh.
- Giảm dimension không cần thiết.

In [3]:
# Tạo df_cleaned từ df_raw
df_cleaned = df_raw.copy()

# Kiểm tra tính unique của id
n_unique = df_cleaned['id'].nunique()
print(f"Unique IDs: {n_unique} / {len(df_cleaned)} rows")
print(f"→ All IDs are unique: {n_unique == len(df_cleaned)}")

# Drop cột id
df_cleaned = df_cleaned.drop(columns=['id'])
print(f"\n✓ Dropped 'id' column")
print(f"Shape after: {df_cleaned.shape}")

Unique IDs: 5110 / 5110 rows
→ All IDs are unique: True

✓ Dropped 'id' column
Shape after: (5110, 11)


## Bước 2: Chuẩn hóa Text cho Categorical Columns
**Purpose:**
- Tránh duplicate category do chênh lệch viết hoa/thường hoặc khoảng trắng.
- Ví dụ: "Male" vs "male" vs " Male " → tất cả thành "male".

**Workflow:**
- Tìm các cột object.
- Áp dụng `.strip().lower()`.
- Giữ NaN đúng format (không biến thành chuỗi "nan").

**Reasoning:**
- Đảm bảo consistency trong categorical encoding.
- Tránh tạo nhiều dummy variables không cần thiết.

In [4]:
# Tìm các cột categorical (object)
cat_cols = df_cleaned.select_dtypes(include=['object']).columns.tolist()
print(f"Categorical columns detected: {cat_cols}")

# Chuẩn hóa: lowercase + strip, giữ NaN
for col in cat_cols:
    df_cleaned[col] = (df_cleaned[col].astype(str)
                                      .str.strip()
                                      .str.lower()
                                      .replace({'nan': np.nan}))

print(f"\n✓ Normalized {len(cat_cols)} categorical columns")

# Hiển thị value counts sau khi normalize
for col in cat_cols:
    print(f"\n{col}:")
    print(df_cleaned[col].value_counts(dropna=False))

Categorical columns detected: ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']

✓ Normalized 5 categorical columns

gender:
gender
female    2994
male      2115
other        1
Name: count, dtype: int64

ever_married:
ever_married
yes    3353
no     1757
Name: count, dtype: int64

work_type:
work_type
private          2925
self-employed     819
children          687
govt_job          657
never_worked       22
Name: count, dtype: int64

Residence_type:
Residence_type
urban    2596
rural    2514
Name: count, dtype: int64

smoking_status:
smoking_status
never smoked       1892
unknown            1544
formerly smoked     885
smokes              789
Name: count, dtype: int64


## Bước 3: Xử lý "unknown" trong smoking_status
**Purpose:**
- Giá trị "unknown" trong `smoking_status` thực chất là **missing disguised**.
- Chuyển "unknown" → NaN để phản ánh đúng bản chất dữ liệu thiếu.

**Workflow:**
- Xem thống kê trước khi thay đổi.
- Replace 'unknown' → NaN.

**Reasoning:**
- Giúp xử lý missing một cách nhất quán.
- Trong modeling, có thể tạo flag hoặc impute hợp lý sau này.

In [5]:
print("BEFORE conversion:")
print(df_cleaned['smoking_status'].value_counts(dropna=False))
print(f"Number of 'unknown': {(df_cleaned['smoking_status'] == 'unknown').sum()}")

# Replace 'unknown' → NaN
df_cleaned['smoking_status'] = df_cleaned['smoking_status'].replace('unknown', np.nan)

print("\nAFTER conversion:")
print(df_cleaned['smoking_status'].value_counts(dropna=False))
print(f"✓ Converted 'unknown' → NaN in smoking_status")

BEFORE conversion:
smoking_status
never smoked       1892
unknown            1544
formerly smoked     885
smokes              789
Name: count, dtype: int64
Number of 'unknown': 1544

AFTER conversion:
smoking_status
never smoked       1892
NaN                1544
formerly smoked     885
smokes              789
Name: count, dtype: int64
✓ Converted 'unknown' → NaN in smoking_status


## Bước 4: Kiểm tra Final Cleaned Data
**Purpose:** Xác nhận kết quả sau preprocessing.

Kiểm tra:
- Shape của dữ liệu
- Missing values
- Sample của dữ liệu đã cleaned

In [6]:
# Final cleaned data summary
print("="*60)
print("CLEANED DATA SUMMARY")
print("="*60)
print(f"Shape: {df_cleaned.shape}")
print(f"\nColumns: {df_cleaned.columns.tolist()}")

# Missing summary
print("\nMissing Values:")
missing_summary = pd.DataFrame({
    "column": df_cleaned.columns,
    "n_missing": df_cleaned.isna().sum().values,
    "pct_missing": (df_cleaned.isna().mean() * 100).round(2).values
})
display(missing_summary[missing_summary['n_missing'] > 0])

# Sample
print("\nSample of cleaned data:")
display(df_cleaned.sample(5, random_state=42))

CLEANED DATA SUMMARY
Shape: (5110, 11)

Columns: ['gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status', 'stroke']

Missing Values:


,column,n_missing,pct_missing
8,bmi,201,3.93
9,smoking_status,1544,30.22



Sample of cleaned data:


,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
4688,male,31.0,0,0,no,self-employed,rural,64.85,23.0,NaN,0
4478,male,40.0,0,0,yes,self-employed,rural,65.29,28.3,never smoked,0
3849,female,8.0,0,0,no,children,urban,74.42,22.5,NaN,0
4355,female,79.0,1,0,yes,self-employed,rural,76.64,19.5,never smoked,0
3826,female,75.0,0,0,yes,govt_job,rural,94.77,27.2,never smoked,0


## Bước 5: Lưu Cleaned Data
**Purpose:** Lưu dữ liệu đã cleaned ra file CSV để sử dụng cho modeling.

**Output:** `healthcare_cleaned.csv` trong thư mục `data/`

In [7]:
# Save cleaned data
output_path = '../data/healthcare_cleaned.csv'
df_cleaned.to_csv(output_path, index=False)

print("="*60)
print("✓ SAVED CLEANED DATA")
print("="*60)
print(f"File: {output_path}")
print(f"Rows: {df_cleaned.shape[0]}")
print(f"Columns: {df_cleaned.shape[1]}")
print("\n✓ Data is ready for modeling!")
print("\nNext steps:")
print("- Imputation for missing values (bmi, smoking_status)")
print("- Feature engineering")
print("- Handling class imbalance")
print("- Scaling/encoding for modeling")

✓ SAVED CLEANED DATA
File: ../data/healthcare_cleaned.csv
Rows: 5110
Columns: 11

✓ Data is ready for modeling!

Next steps:
- Imputation for missing values (bmi, smoking_status)
- Feature engineering
- Handling class imbalance
- Scaling/encoding for modeling
